**Instalation de bilbliothéque**

In [60]:
# Installation de YOLO
#!pip install ultralytics
import time
from ultralytics import YOLO
import os
import cv2
import shutil
import numpy as np

In [80]:

baseline_folder = './dataset2012/dataset'
# Chemin du nouveau dataset
dataset_folder = './MoCA_Video'  # Update this path
train_folder = os.path.join(dataset_folder, 'TrainDataset_per_sq')
test_folder = os.path.join(dataset_folder, 'TestDataset_per_sq')

results_base_dir = os.path.join(dataset_folder, 'results')
os.makedirs(results_base_dir, exist_ok=True)


In [81]:
model = YOLO('yolo11n-seg.pt')
# Dossiers des vidéos
videos = ['PETS2006', 'highway', 'office', 'pedestrians']
TARGET_SUBFOLDERS = ['hedgehog_0', 'stick_insect_0']

# Dossier pour les résultats
results_dir = os.path.join(baseline_folder, 'results')
os.makedirs(results_dir, exist_ok=True)

In [64]:
# Fonction pour détecter les objets dans une image
def detect_objects(image_path, conf_threshold=0.5):
    # Charger l'image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Erreur : Impossible de charger l'image {image_path}")
        return

    # Détecter les objets avec YOLO
    results = model(image_path, conf=conf_threshold)

    # Afficher les résultats
    for result in results:
        # Visualiser les détections sur l'image
        annotated_img = result.plot()  # Dessine les boîtes englobantes et les étiquettes

        # Sauvegarder l'image annotée
        output_path = os.path.join(results_dir, os.path.basename(image_path).replace('.jpg', '_detection.jpg'))
        cv2.imwrite(output_path, annotated_img)
        print(f"Image annotée sauvegardée à : {output_path}")

In [65]:
#Petits tests sur image solo pour vérifier que ça marche bien avant

image_path = os.path.join(baseline_folder, 'baseline', 'office', 'input', 'in001922.jpg')
#image_path = os.path.join(baseline_dir, 'baseline', 'PETS2006', 'input', 'in000994.jpg')
#image_path = os.path.join(baseline_dir, 'baseline', 'highway', 'input', 'in000829.jpg')
# Traiter l'image
def process_image(image_path, conf_threshold=0.5):
    # Charger l'image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Erreur : Impossible de charger l'image {image_path}")
        return

    # Appliquer YOLO avec un seuil de confiance
    results = model(image_path, conf=conf_threshold)

    # Initialiser une image vide pour le masque
    height, width = img.shape[:2]
    mask_img = np.zeros((height, width), dtype=np.uint8)

    # Extraire et combiner les masques
    if results[0].masks is not None:
        masks = results[0].masks.data.cpu().numpy()

        for mask in masks:
            mask = (mask > 0.5).astype(np.uint8) * 255  # Seuillage binaire
            mask_resized = cv2.resize(mask, (width, height))
            mask_img = np.maximum(mask_img, mask_resized)  # Fusionner les masques

        # Sauvegarder le masque binaire
        output_path = os.path.join(results_dir, os.path.basename(image_path).replace('.jpg', '_mask.png'))
        cv2.imwrite(output_path, mask_img)
        print(f"Masque sauvegardé à : {output_path}")


# Tester l'image
#process_image(image_path)



In [66]:
# Fonction pour traiter une vidéo et afficher les objets détectés
def process_and_show_video(video_name):
    input_dir = os.path.join(baseline_folder, 'baseline', video_name, 'input')
    output_dir = os.path.join(results_dir, video_name)
    os.makedirs(output_dir, exist_ok=True)

    # Lire les bornes temporelles
    temporal_roi_path = os.path.join(baseline_folder, 'baseline', video_name, 'temporalROI.txt')
    with open(temporal_roi_path, 'r') as file:
        start_frame, end_frame = map(int, file.readline().strip().split())

    # Lister les images d'entrée
    frames = sorted([f for f in os.listdir(input_dir) if f.endswith('.jpg')])

    for frame in frames:
        frame_number = int(''.join(filter(str.isdigit, frame)))
        if frame_number < start_frame or frame_number > end_frame:
            continue

        img_path = os.path.join(input_dir, frame)

        # Détecter les objets dans l'image
        results = model(img_path, verbose=False)

        # Visualiser les résultats
        for result in results:
            annotated_frame = result.plot()  # Dessiner les boîtes englobantes et les étiquettes

            # Afficher l'image annotée
            cv2.imshow(f'Detected Objects - {video_name}', annotated_frame)
            cv2.waitKey(1)  # Attendre 1 ms entre les frames

            # Sauvegarder l'image annotée
            output_path = os.path.join(output_dir, frame.replace('in', 'out').replace('.jpg', '.jpg'))
            cv2.imwrite(output_path, annotated_frame)

    # Fermer la fenêtre OpenCV
    cv2.destroyAllWindows()

In [67]:

# Traiter toutes les vidéos
start_time = time.time()
for video in videos:
    process_and_show_video(video)
end_time = time.time()

print(f"Temps total de traitement : {end_time - start_time:.2f} secondes")

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
import gc


def evaluate_results():
    for video_name in videos:
        print(f"\nÉvaluation pour la vidéo : {video_name}")

        gt_dir = os.path.join(baseline_folder, 'baseline', video_name, 'groundtruth')
        res_dir = os.path.join(results_dir, video_name)

        gt_frames = sorted([f for f in os.listdir(gt_dir) if f.endswith('.png')])
        res_frames = sorted([f for f in os.listdir(res_dir) if f.endswith('.png')])

        y_true = []
        y_pred = []

        total_iou = 0
        num_frames = 0

        for idx, (gt_frame, res_frame) in enumerate(zip(gt_frames, res_frames)):
            # Filtrer tous les 10 images
            if idx % 10 != 0:
                continue

            # Vérifier si les fichiers existent
            if not os.path.exists(os.path.join(gt_dir, gt_frame)) or not os.path.exists(os.path.join(res_dir, res_frame)):
                print(f"Fichier manquant : {gt_frame} ou {res_frame}")
                continue

            # Charger les images
            gt_img = cv2.imread(os.path.join(gt_dir, gt_frame), cv2.IMREAD_GRAYSCALE)
            res_img = cv2.imread(os.path.join(res_dir, res_frame), cv2.IMREAD_GRAYSCALE)

            # économiser la mémoire
            gt_img = cv2.resize(gt_img, (320, 240))
            res_img = cv2.resize(res_img, (320, 240))

            # Binarisation
            gt_img = (gt_img > 0).astype(int)
            res_img = (res_img > 0).astype(int)

            # Calcul IoU avec jaccard_score
            iou = jaccard_score(gt_img.flatten(), res_img.flatten())
            total_iou += iou
            num_frames += 1

            # Pour les métriques classiques
            y_true.extend(gt_img.flatten())
            y_pred.extend(res_img.flatten())

            # Libération de la mémoire
            del gt_img, res_img
            gc.collect()

        # Calcul des métriques
        precision = precision_score(y_true, y_pred, zero_division=1)
        recall = recall_score(y_true, y_pred, zero_division=1)
        f1 = f1_score(y_true, y_pred, zero_division=1)
        mean_iou = total_iou / num_frames if num_frames > 0 else 0

        # Affichage des résultats
        print(f"{'Métrique':<15} {'Valeur':<10}")
        print(f"{'Précision':<15} {precision:.2f}")
        print(f"{'Rappel':<15} {recall:.2f}")
        print(f"{'F1-score':<15} {f1:.2f}")
        print(f"{'IoU moyen':<15} {mean_iou:.2f}")

evaluate_results()


Évaluation pour la vidéo : PETS2006
Métrique        Valeur    
Précision       0.33
Rappel          0.21
F1-score        0.26
IoU moyen       0.08

Évaluation pour la vidéo : highway
Métrique        Valeur    
Précision       0.47
Rappel          0.08
F1-score        0.14
IoU moyen       0.09

Évaluation pour la vidéo : office
Métrique        Valeur    
Précision       0.69
Rappel          0.20
F1-score        0.31
IoU moyen       0.31

Évaluation pour la vidéo : pedestrians
Métrique        Valeur    
Précision       0.46
Rappel          0.05
F1-score        0.09
IoU moyen       0.03


In [83]:
def process_subfolder(subfolder_path, output_path, conf_threshold=0.5):
    """
    For a given subfolder (e.g. 'hedgehog_0'), run YOLO segmentation on all .jpg images
    in the 'Imgs' subdirectory, and save binary masks in `output_path`.
    """
    imgs_dir = os.path.join(subfolder_path, 'Imgs')
    if not os.path.exists(imgs_dir):
        print(f"No 'Imgs' folder found in {subfolder_path}")
        return

    os.makedirs(output_path, exist_ok=True)

    # List all .jpg images
    image_files = sorted([f for f in os.listdir(imgs_dir) if f.lower().endswith('.jpg')])

    for img_name in image_files:
        img_path = os.path.join(imgs_dir, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error reading image: {img_path}")
            continue

        # Run inference
        results = model(img_path, conf=conf_threshold)

        # Prepare a blank mask
        height, width = img.shape[:2]
        mask_img = np.zeros((height, width), dtype=np.uint8)

        # If segmentation masks exist, combine them
        if results[0].masks is not None:
            masks = results[0].masks.data.cpu().numpy()
            for mask in masks:
                mask_bin = (mask > 0.5).astype(np.uint8) * 255
                mask_resized = cv2.resize(mask_bin, (width, height))
                mask_img = np.maximum(mask_img, mask_resized)

        # Save the mask (replace .jpg with _mask.png)
        mask_filename = os.path.splitext(img_name)[0] + '_mask.png'
        mask_path = os.path.join(output_path, mask_filename)
        cv2.imwrite(mask_path, mask_img)

    print(f"Done processing subfolder: {subfolder_path}. Masks saved in: {output_path}")

# ------------------------------------------------------------------------------
# 3. Function: Evaluate masks against GT for a subfolder
# ------------------------------------------------------------------------------
def evaluate_subfolder(subfolder_path, results_subfolder_path):
    """
    Compare the generated masks in `results_subfolder_path` with the ground-truth
    images in `GT/` inside `subfolder_path`.
    """
    gt_dir = os.path.join(subfolder_path, 'GT')
    if not os.path.exists(gt_dir):
        print(f"No 'GT' folder found in {subfolder_path}. Skipping evaluation.")
        return

    # List ground-truth (.png) and predicted mask (.png) files
    gt_files = sorted([f for f in os.listdir(gt_dir) if f.lower().endswith('.png')])
    pred_files = sorted([f for f in os.listdir(results_subfolder_path) if f.lower().endswith('.png')])

    if not gt_files or not pred_files:
        print("No .png files found for GT or predictions. Skipping.")
        return

    y_true = []
    y_pred = []
    total_iou = 0.0
    num_frames = 0

    for gt_name in gt_files:
        # We assume your predicted mask has the same name with '_mask' appended
        # e.g., 00000.png -> 00000_mask.png
        base_name = os.path.splitext(gt_name)[0]  
        pred_name = base_name + '_mask.png'

        gt_path = os.path.join(gt_dir, gt_name)
        pred_path = os.path.join(results_subfolder_path, pred_name)

        if not os.path.exists(pred_path):
            continue  # No matching prediction

        # Load images in grayscale
        gt_img = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
        pr_img = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)

        if gt_img is None or pr_img is None:
            continue

        # Binarize
        gt_bin = (gt_img > 0).astype(np.uint8)
        pr_bin = (pr_img > 0).astype(np.uint8)

        # IoU (Jaccard)
        iou = jaccard_score(gt_bin.flatten(), pr_bin.flatten())
        total_iou += iou
        num_frames += 1

        # For precision/recall
        y_true.extend(gt_bin.flatten())
        y_pred.extend(pr_bin.flatten())

        # Free memory
        del gt_img, pr_img
        gc.collect()

    if num_frames == 0:
        print("No matching frames for evaluation.")
        return

    precision = precision_score(y_true, y_pred, zero_division=1)
    recall    = recall_score(y_true, y_pred, zero_division=1)
    f1        = f1_score(y_true, y_pred, zero_division=1)
    mean_iou  = total_iou / num_frames

    print(f"Evaluation for: {subfolder_path}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    print(f"  F1-Score:  {f1:.3f}")
    print(f"  Mean IoU:  {mean_iou:.3f}")

# ------------------------------------------------------------------------------
# 4. Main script to process only hedgehog_0 and stick_insect_0
# ------------------------------------------------------------------------------
def main_process_and_evaluate(base_folder, set_name="Train"):
    # Directory to save the segmentation results for this set
    results_set_dir = os.path.join(results_base_dir, set_name)
    os.makedirs(results_set_dir, exist_ok=True)

    # We ONLY process these two subfolders
    for subfolder_name in TARGET_SUBFOLDERS:
        subfolder_path = os.path.join(base_folder, subfolder_name)
        if not os.path.isdir(subfolder_path):
            print(f"Subfolder not found: {subfolder_path}. Skipping.")
            continue

        print(f"\n--- Processing subfolder: {subfolder_name} ---")

        # 1) Run detection + generate masks
        output_subfolder_path = os.path.join(results_set_dir, subfolder_name)
        start_time = time.time()
        process_subfolder(subfolder_path, output_subfolder_path, conf_threshold=0.5)
        end_time = time.time()
        print(f"  Detection time: {end_time - start_time:.2f} s")

        # 2) Evaluate results
        evaluate_subfolder(subfolder_path, output_subfolder_path)

# ------------------------------------------------------------------------------
# 5. Run for both Train and Test sets (if desired)
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    print("Processing TRAIN set for hedgehog_0 and stick_insect_0...")
    main_process_and_evaluate(train_folder, set_name="Train")

    

    print("\nAll done!")

Processing TRAIN set for hedgehog_0 and stick_insect_0...

--- Processing subfolder: hedgehog_0 ---



image 1/1 c:\Users\war machine\Desktop\INF6804\TP2\MoCA_Video\TrainDataset_per_sq\hedgehog_0\Imgs\00000.jpg: 384x640 (no detections), 116.8ms
Speed: 1.8ms preprocess, 116.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 c:\Users\war machine\Desktop\INF6804\TP2\MoCA_Video\TrainDataset_per_sq\hedgehog_0\Imgs\00005.jpg: 384x640 (no detections), 107.8ms
Speed: 2.7ms preprocess, 107.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 c:\Users\war machine\Desktop\INF6804\TP2\MoCA_Video\TrainDataset_per_sq\hedgehog_0\Imgs\00010.jpg: 384x640 (no detections), 104.6ms
Speed: 2.2ms preprocess, 104.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 c:\Users\war machine\Desktop\INF6804\TP2\MoCA_Video\TrainDataset_per_sq\hedgehog_0\Imgs\00015.jpg: 384x640 (no detections), 113.8ms
Speed: 1.7ms preprocess, 113.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 c:\Users\war machine\Deskt